# Machine Learning Regression Models

**DOST-ITDI AI Training Workshop**  
**Day 1 - Session 3: Machine Learning with Scikit-learn**

---

## Learning Objectives
1. Understand regression problems in chemistry
2. Implement various regression algorithms
3. Evaluate model performance using metrics
4. Compare different models
5. Apply feature importance analysis

## What is Regression?

Regression is a supervised learning technique used to predict **continuous values**.

**Chemistry Applications**:
- Predicting molecular properties (boiling point, solubility, toxicity)
- Estimating reaction yields
- Forecasting material properties
- QSAR (Quantitative Structure-Activity Relationship) modeling

## 1. Setup and Data Loading

In [ ]:
# Install required libraries
!pip install rdkit scikit-learn seaborn -q

print("✓ Installation complete!")

In [ ]:
# Import libraries
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from rdkit import Chem
from rdkit.Chem import Descriptors
import warnings
warnings.filterwarnings('ignore')

# Machine Learning
from sklearn.model_selection import train_test_split, cross_val_score, GridSearchCV
from sklearn.preprocessing import StandardScaler
from sklearn.linear_model import LinearRegression, Ridge, Lasso
from sklearn.tree import DecisionTreeRegressor
from sklearn.ensemble import RandomForestRegressor, GradientBoostingRegressor
from sklearn.svm import SVR
from sklearn.metrics import mean_squared_error, mean_absolute_error, r2_score

# Set random seed for reproducibility
np.random.seed(42)

# Plotting style
sns.set_style("whitegrid")
plt.rcParams['figure.figsize'] = (10, 6)

print("✓ Libraries imported successfully!")

In [ ]:
# Load ESOL dataset
url = "https://raw.githubusercontent.com/deepchem/deepchem/master/datasets/delaney-processed.csv"
df = pd.read_csv(url)

print(f"Dataset shape: {df.shape}")
print(f"\nTarget variable: measured log solubility in mols per litre")
print(f"Range: {df['measured log solubility in mols per litre'].min():.2f} to {df['measured log solubility in mols per litre'].max():.2f}")
df.head()

## 2. Data Preparation

In [ ]:
# Compute additional descriptors
df['mol'] = df['smiles'].apply(Chem.MolFromSmiles)
df = df[df['mol'].notna()].copy()  # Remove invalid SMILES

df['LogP'] = df['mol'].apply(Descriptors.MolLogP)
df['NumHAcceptors'] = df['mol'].apply(Descriptors.NumHAcceptors)
df['NumAromaticRings'] = df['mol'].apply(Descriptors.NumAromaticRings)
df['TPSA'] = df['mol'].apply(Descriptors.TPSA)

print(f"Dataset after processing: {df.shape[0]} molecules")

In [ ]:
# Select features
feature_columns = [
    'Molecular Weight',
    'Number of H-Bond Donors',
    'Number of Rings',
    'Number of Rotatable Bonds',
    'Polar Surface Area',
    'LogP',
    'NumHAcceptors',
    'NumAromaticRings'
]

X = df[feature_columns]
y = df['measured log solubility in mols per litre']

print("Features:")
print(X.head())
print(f"\nFeatures shape: {X.shape}")
print(f"Target shape: {y.shape}")

In [ ]:
# Split data: 80% training, 20% testing
X_train, X_test, y_train, y_test = train_test_split(
    X, y, test_size=0.2, random_state=42
)

print("Dataset Split:")
print(f"Training set: {X_train.shape[0]} samples ({X_train.shape[0]/len(X)*100:.1f}%)")
print(f"Test set: {X_test.shape[0]} samples ({X_test.shape[0]/len(X)*100:.1f}%)")

In [ ]:
# Feature scaling (standardization)
scaler = StandardScaler()
X_train_scaled = scaler.fit_transform(X_train)
X_test_scaled = scaler.transform(X_test)

print("Feature scaling complete!")
print(f"Mean of scaled training features: {X_train_scaled.mean(axis=0).round(10)}")
print(f"Std of scaled training features: {X_train_scaled.std(axis=0).round(2)}")

## 3. Regression Models

### 3.1 Linear Regression

The simplest regression model: finds the best-fit line through the data.

In [ ]:
# Train Linear Regression
lr = LinearRegression()
lr.fit(X_train_scaled, y_train)

# Make predictions
y_train_pred_lr = lr.predict(X_train_scaled)
y_test_pred_lr = lr.predict(X_test_scaled)

# Evaluate
print("Linear Regression Results:")
print(f"Training R²: {r2_score(y_train, y_train_pred_lr):.4f}")
print(f"Test R²: {r2_score(y_test, y_test_pred_lr):.4f}")
print(f"Test RMSE: {np.sqrt(mean_squared_error(y_test, y_test_pred_lr)):.4f}")
print(f"Test MAE: {mean_absolute_error(y_test, y_test_pred_lr):.4f}")

In [ ]:
# Feature importance (coefficients)
feature_importance = pd.DataFrame({
    'Feature': feature_columns,
    'Coefficient': lr.coef_
}).sort_values('Coefficient', key=abs, ascending=False)

plt.figure(figsize=(10, 6))
colors = ['red' if x < 0 else 'green' for x in feature_importance['Coefficient']]
plt.barh(feature_importance['Feature'], feature_importance['Coefficient'],
         color=colors, alpha=0.7, edgecolor='black')
plt.xlabel('Coefficient Value', fontsize=12)
plt.title('Linear Regression: Feature Importance', fontsize=14, fontweight='bold')
plt.axvline(0, color='black', linewidth=0.8)
plt.grid(True, alpha=0.3, axis='x')
plt.tight_layout()
plt.show()

### 3.2 Ridge Regression (L2 Regularization)

Prevents overfitting by penalizing large coefficients.

In [ ]:
# Train Ridge Regression
ridge = Ridge(alpha=1.0, random_state=42)
ridge.fit(X_train_scaled, y_train)

# Predictions
y_train_pred_ridge = ridge.predict(X_train_scaled)
y_test_pred_ridge = ridge.predict(X_test_scaled)

# Evaluate
print("Ridge Regression Results:")
print(f"Training R²: {r2_score(y_train, y_train_pred_ridge):.4f}")
print(f"Test R²: {r2_score(y_test, y_test_pred_ridge):.4f}")
print(f"Test RMSE: {np.sqrt(mean_squared_error(y_test, y_test_pred_ridge)):.4f}")
print(f"Test MAE: {mean_absolute_error(y_test, y_test_pred_ridge):.4f}")

### 3.3 Random Forest Regression

Ensemble of decision trees - powerful and versatile.

In [ ]:
# Train Random Forest
rf = RandomForestRegressor(n_estimators=100, random_state=42, n_jobs=-1)
rf.fit(X_train_scaled, y_train)

# Predictions
y_train_pred_rf = rf.predict(X_train_scaled)
y_test_pred_rf = rf.predict(X_test_scaled)

# Evaluate
print("Random Forest Results:")
print(f"Training R²: {r2_score(y_train, y_train_pred_rf):.4f}")
print(f"Test R²: {r2_score(y_test, y_test_pred_rf):.4f}")
print(f"Test RMSE: {np.sqrt(mean_squared_error(y_test, y_test_pred_rf)):.4f}")
print(f"Test MAE: {mean_absolute_error(y_test, y_test_pred_rf):.4f}")

In [ ]:
# Random Forest Feature Importance
rf_importance = pd.DataFrame({
    'Feature': feature_columns,
    'Importance': rf.feature_importances_
}).sort_values('Importance', ascending=False)

plt.figure(figsize=(10, 6))
plt.barh(rf_importance['Feature'], rf_importance['Importance'],
         color='steelblue', alpha=0.7, edgecolor='black')
plt.xlabel('Importance Score', fontsize=12)
plt.title('Random Forest: Feature Importance', fontsize=14, fontweight='bold')
plt.grid(True, alpha=0.3, axis='x')
plt.tight_layout()
plt.show()

### 3.4 Gradient Boosting Regression

Sequential ensemble method - builds trees to correct previous errors.

In [ ]:
# Train Gradient Boosting
gbr = GradientBoostingRegressor(n_estimators=100, learning_rate=0.1,
                                 max_depth=3, random_state=42)
gbr.fit(X_train_scaled, y_train)

# Predictions
y_train_pred_gbr = gbr.predict(X_train_scaled)
y_test_pred_gbr = gbr.predict(X_test_scaled)

# Evaluate
print("Gradient Boosting Results:")
print(f"Training R²: {r2_score(y_train, y_train_pred_gbr):.4f}")
print(f"Test R²: {r2_score(y_test, y_test_pred_gbr):.4f}")
print(f"Test RMSE: {np.sqrt(mean_squared_error(y_test, y_test_pred_gbr)):.4f}")
print(f"Test MAE: {mean_absolute_error(y_test, y_test_pred_gbr):.4f}")

## 4. Model Comparison

In [ ]:
# Compile results
results = pd.DataFrame({
    'Model': ['Linear Regression', 'Ridge', 'Random Forest', 'Gradient Boosting'],
    'Train R²': [
        r2_score(y_train, y_train_pred_lr),
        r2_score(y_train, y_train_pred_ridge),
        r2_score(y_train, y_train_pred_rf),
        r2_score(y_train, y_train_pred_gbr)
    ],
    'Test R²': [
        r2_score(y_test, y_test_pred_lr),
        r2_score(y_test, y_test_pred_ridge),
        r2_score(y_test, y_test_pred_rf),
        r2_score(y_test, y_test_pred_gbr)
    ],
    'Test RMSE': [
        np.sqrt(mean_squared_error(y_test, y_test_pred_lr)),
        np.sqrt(mean_squared_error(y_test, y_test_pred_ridge)),
        np.sqrt(mean_squared_error(y_test, y_test_pred_rf)),
        np.sqrt(mean_squared_error(y_test, y_test_pred_gbr))
    ],
    'Test MAE': [
        mean_absolute_error(y_test, y_test_pred_lr),
        mean_absolute_error(y_test, y_test_pred_ridge),
        mean_absolute_error(y_test, y_test_pred_rf),
        mean_absolute_error(y_test, y_test_pred_gbr)
    ]
})

print("\nModel Performance Comparison:")
print(results.to_string(index=False))

# Find best model
best_model_idx = results['Test R²'].idxmax()
print(f"\n🏆 Best Model: {results.loc[best_model_idx, 'Model']}")
print(f"   Test R²: {results.loc[best_model_idx, 'Test R²']:.4f}")

In [ ]:
# Visualize model comparison
fig, axes = plt.subplots(1, 2, figsize=(15, 5))

# R² Score comparison
x = np.arange(len(results))
width = 0.35

axes[0].bar(x - width/2, results['Train R²'], width, label='Train',
            alpha=0.8, edgecolor='black')
axes[0].bar(x + width/2, results['Test R²'], width, label='Test',
            alpha=0.8, edgecolor='black')
axes[0].set_xlabel('Model', fontsize=12)
axes[0].set_ylabel('R² Score', fontsize=12)
axes[0].set_title('R² Score Comparison', fontsize=13, fontweight='bold')
axes[0].set_xticks(x)
axes[0].set_xticklabels(results['Model'], rotation=45, ha='right')
axes[0].legend()
axes[0].grid(True, alpha=0.3, axis='y')

# RMSE comparison
axes[1].bar(results['Model'], results['Test RMSE'],
            color='coral', alpha=0.8, edgecolor='black')
axes[1].set_xlabel('Model', fontsize=12)
axes[1].set_ylabel('RMSE', fontsize=12)
axes[1].set_title('Test RMSE Comparison (Lower is Better)', fontsize=13, fontweight='bold')
axes[1].set_xticklabels(results['Model'], rotation=45, ha='right')
axes[1].grid(True, alpha=0.3, axis='y')

plt.tight_layout()
plt.show()

## 5. Prediction Visualization

In [ ]:
# Actual vs Predicted plots
fig, axes = plt.subplots(2, 2, figsize=(14, 12))
axes = axes.ravel()

predictions = [
    ('Linear Regression', y_test_pred_lr),
    ('Ridge', y_test_pred_ridge),
    ('Random Forest', y_test_pred_rf),
    ('Gradient Boosting', y_test_pred_gbr)
]

for idx, (name, y_pred) in enumerate(predictions):
    axes[idx].scatter(y_test, y_pred, alpha=0.5, s=50, edgecolors='black', linewidth=0.5)

    # Perfect prediction line
    min_val = min(y_test.min(), y_pred.min())
    max_val = max(y_test.max(), y_pred.max())
    axes[idx].plot([min_val, max_val], [min_val, max_val], 'r--', linewidth=2, label='Perfect Prediction')

    axes[idx].set_xlabel('Actual Log Solubility', fontsize=11)
    axes[idx].set_ylabel('Predicted Log Solubility', fontsize=11)
    axes[idx].set_title(f'{name}', fontsize=12, fontweight='bold')
    axes[idx].grid(True, alpha=0.3)
    axes[idx].legend()

    # Add R² score
    r2 = r2_score(y_test, y_pred)
    axes[idx].text(0.05, 0.95, f'R² = {r2:.4f}', transform=axes[idx].transAxes,
                  fontsize=10, verticalalignment='top',
                  bbox=dict(boxstyle='round', facecolor='wheat', alpha=0.7))

plt.tight_layout()
plt.show()

## 6. Residual Analysis

In [ ]:
# Residuals = Actual - Predicted
residuals_rf = y_test - y_test_pred_rf

fig, axes = plt.subplots(1, 2, figsize=(15, 5))

# Residual plot
axes[0].scatter(y_test_pred_rf, residuals_rf, alpha=0.5, s=50,
                edgecolors='black', linewidth=0.5)
axes[0].axhline(y=0, color='r', linestyle='--', linewidth=2)
axes[0].set_xlabel('Predicted Log Solubility', fontsize=12)
axes[0].set_ylabel('Residuals', fontsize=12)
axes[0].set_title('Residual Plot (Random Forest)', fontsize=13, fontweight='bold')
axes[0].grid(True, alpha=0.3)

# Residual distribution
axes[1].hist(residuals_rf, bins=30, edgecolor='black', alpha=0.7, color='steelblue')
axes[1].axvline(x=0, color='r', linestyle='--', linewidth=2)
axes[1].set_xlabel('Residuals', fontsize=12)
axes[1].set_ylabel('Frequency', fontsize=12)
axes[1].set_title('Distribution of Residuals', fontsize=13, fontweight='bold')
axes[1].grid(True, alpha=0.3, axis='y')

plt.tight_layout()
plt.show()

print(f"Mean Residual: {residuals_rf.mean():.6f}")
print(f"Std Dev of Residuals: {residuals_rf.std():.4f}")

## 7. Cross-Validation

More robust evaluation using multiple train/test splits.

In [ ]:
# 5-Fold Cross-Validation
models = {
    'Linear Regression': LinearRegression(),
    'Ridge': Ridge(alpha=1.0),
    'Random Forest': RandomForestRegressor(n_estimators=100, random_state=42),
    'Gradient Boosting': GradientBoostingRegressor(n_estimators=100, random_state=42)
}

cv_results = {}

print("Cross-Validation Results (5-Fold):")
print("=" * 60)

for name, model in models.items():
    scores = cross_val_score(model, X_train_scaled, y_train,
                            cv=5, scoring='r2', n_jobs=-1)
    cv_results[name] = scores
    print(f"{name:20} | Mean R²: {scores.mean():.4f} (+/- {scores.std():.4f})")

In [ ]:
# Visualize cross-validation results
plt.figure(figsize=(10, 6))
plt.boxplot(cv_results.values(), labels=cv_results.keys())
plt.ylabel('R² Score', fontsize=12)
plt.title('Cross-Validation R² Scores (5-Fold)', fontsize=14, fontweight='bold')
plt.xticks(rotation=45, ha='right')
plt.grid(True, alpha=0.3, axis='y')
plt.tight_layout()
plt.show()

## 8. Making Predictions on New Data

In [ ]:
# Example: Predict solubility for a new molecule
# Let's use the best model (Random Forest)

# Take a few examples from test set
sample_indices = [0, 10, 20, 30, 40]
samples = X_test.iloc[sample_indices]
samples_scaled = scaler.transform(samples)

# Predict
predictions = rf.predict(samples_scaled)
actual = y_test.iloc[sample_indices].values

# Display results
results_df = pd.DataFrame({
    'Actual': actual,
    'Predicted': predictions,
    'Error': actual - predictions,
    'Abs Error': np.abs(actual - predictions)
})

print("\nSample Predictions:")
print(results_df.to_string())
print(f"\nMean Absolute Error: {results_df['Abs Error'].mean():.4f}")

## 9. Understanding Regression Metrics

### Key Metrics:

1. **R² Score (Coefficient of Determination)**
   - Range: -∞ to 1 (1 is perfect)
   - Proportion of variance explained by the model
   - Higher is better

2. **RMSE (Root Mean Squared Error)**
   - Same units as target variable
   - Penalizes large errors more
   - Lower is better

3. **MAE (Mean Absolute Error)**
   - Average absolute error
   - More robust to outliers than RMSE
   - Lower is better

## 10. Summary and Key Takeaways

### What We Learned:

1. **Regression Problem Setup**
   - Feature selection and engineering
   - Data splitting (train/test)
   - Feature scaling

2. **Multiple Regression Models**
   - Linear Regression (baseline)
   - Ridge (regularized linear)
   - Random Forest (ensemble)
   - Gradient Boosting (sequential ensemble)

3. **Model Evaluation**
   - R², RMSE, MAE metrics
   - Cross-validation
   - Residual analysis

4. **Best Practices**
   - Always use train/test split
   - Scale features for linear models
   - Compare multiple models
   - Use cross-validation for robust evaluation
   - Analyze residuals to check model assumptions

### Chemistry Insights:
- LogP (lipophilicity) is a strong predictor of solubility
- Molecular weight negatively correlates with solubility
- Ensemble methods (RF, GBR) often outperform linear models
- Feature importance reveals key molecular properties

## 11. Exercise: Build Your Own Model

In [ ]:
# TODO: Try different regression models or parameters
# 1. Try SVR (Support Vector Regression)
# 2. Try different Random Forest parameters (n_estimators, max_depth)
# 3. Try XGBoost if installed
# 4. Compare your results with the models above

# Your code here:


---

## Resources

- [Scikit-learn Regression Guide](https://scikit-learn.org/stable/supervised_learning.html#supervised-learning)
- [Understanding R² Score](https://en.wikipedia.org/wiki/Coefficient_of_determination)
- [QSAR Modeling](https://en.wikipedia.org/wiki/Quantitative_structure%E2%80%93activity_relationship)

**Next Notebook: Classification Models for Molecular Data**